# Augmentations

In [0]:
# -*- coding: utf-8 -*-
from PIL import ImageFont
from PIL import Image, ImageEnhance
from PIL import ImageDraw
from numpy import random
import itertools
import numpy as np

import torch
from torchvision import transforms
import os
from math import *
import cv2
from scipy import ndimage




erosion_kernels = []
# erosion_kernels.append(np.ones((3, 3), np.uint8))
erosion_kernels.append(np.ones((2, 2), np.uint8))
# erosion_kernels.append(np.ones((1, 3), np.uint8))
erosion_kernels.append(np.ones((1, 2), np.uint8))
# erosion_kernels.append(np.ones((3, 1), np.uint8))
erosion_kernels.append(np.ones((2, 1), np.uint8))




def affine_transform(image, p=0.5):
    if random.random() < p:
        height, width = image.shape[:2]

        pts1 = np.float32([[0, 0], [100, 0], [0, 100]])
        pts2 = np.float32([[0, 0], [100, 0], [random.randint(-15, 15), 100]])

        matrix = cv2.getAffineTransform(pts1, pts2)

        image = cv2.warpAffine(image, matrix, (width, height), flags=cv2.INTER_LINEAR,
                               borderMode=cv2.BORDER_REFLECT_101,
                               borderValue=0)

    return image

def warping(img, warpCoeff, p=0.5):
    if random.random() < p:
        newX, newY = int(img.shape[1] * warpCoeff), img.shape[0]

        pts1 = np.float32([[0, 0], [100, 0], [0, 100]])
        pts2 = np.float32([[0, 0], [100*warpCoeff, 0], [0, 100]])

        matrix = cv2.getAffineTransform(pts1, pts2)

        img = cv2.warpAffine(img, matrix, dsize=(newX, newY))

    return img


def elastic_transform(image, alpha, sigma, border_mode=cv2.BORDER_REFLECT_101, approximate=False, p=0.5):
    if random.random() < p:
        #random_state = np.random.RandomState(1234)

        height, width = image.shape[:2]

        if approximate:
            # Approximate computation smooth displacement map with a large enough kernel.
            # On large images (512+) this is approximately 2X times faster
            dx = (random_state.rand(height, width).astype(np.float32) * 2 - 1)
            cv2.GaussianBlur(dx, (17, 17), sigma, dst=dx)
            dx *= alpha

            dy = (random_state.rand(height, width).astype(np.float32) * 2 - 1)
            cv2.GaussianBlur(dy, (17, 17), sigma, dst=dy)
            dy *= alpha
        else:
            dx = np.float32(ndimage.gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma) * alpha)
            dy = np.float32(ndimage.gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma) * alpha)

            # dx = np.ones((height, width), dtype=np.float32)*50

        x, y = np.meshgrid(np.arange(width), np.arange(height))

        mapx = np.float32(x + dx)
        mapy = np.float32(y + dy)

        return cv2.remap(image, mapx, mapy, cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)

    return image


def speckle(img, p=0.5):
    if random.random() < p:
        severity = np.random.uniform(5, 30)
        blur = ndimage.gaussian_filter(np.random.randn(img.shape[0], img.shape[1]) * severity, 1)
        img = img.astype(np.float32)

        img[:, :, 0] += blur
        img[:, :, 1] += blur
        img[:, :, 2] += blur

        img[img > 255] = 255
        img[img <= 0] = 0
        img = img.astype(np.uint8)

    return img


def gen_mask(h, w):
    severity = np.random.uniform(5, 50)
    blur = ndimage.gaussian_filter(np.random.randn(h, w) * severity + 188, 1)

    blur[blur > 255] = 255
    blur[blur <= 0] = 0
    blur = blur.astype(np.uint8)

    return blur


def erode_image(img, kernel='auto', p=0.5):
    if random.random() < p:
        # kernel_size = random.randint(low=2, high=4)
        # kernel = np.ones((kernel_size, kernel_size), np.uint8)

        if isinstance(kernel, str) and kernel == 'auto':
            erosion_kernel = erosion_kernels[random.randint(0, len(erosion_kernels))]
        else:
            erosion_kernel = kernel

        img[:, :, 3] = (255 - img[:, :, 3])
        img = cv2.erode(img.astype(np.float32) / 255.0, erosion_kernel, iterations=1)
        img = (img * 255).astype(np.uint8)
        img[:, :, 3] = (255 - img[:, :, 3])

    return img


def blur(img, ksize, p = 0.5):
    if random.random() < p * 0.5:
        return cv2.blur(img, (ksize//2, ksize//2))
    if random.random() < p:
        return cv2.GaussianBlur(img, (ksize, ksize), sigmaX=0)

    return img


def gen_mask_2(h, w):
    n_holes = random.randint(4, 50)
    mask = np.ones((h, w), np.float32) * random.randint(150, 200)

    for n in range(n_holes):
        y = np.random.randint(h)
        x = np.random.randint(w)

        hole_size = random.randint(10, 50)
        y1 = np.clip(y - hole_size // 2, 0, h)
        y2 = np.clip(y + hole_size // 2, 0, h)
        x1 = np.clip(x - hole_size // 2, 0, w)
        x2 = np.clip(x + hole_size // 2, 0, w)

        mask[y1: y2, x1: x2] = random.randint(50, 220)

    # img = Blur(blur_limit=10, p=1.0)(image=mask)['image']
    # img = ndimage.gaussian_filter(mask, 5)
    img = ndimage.uniform_filter(mask, size=15)

    img = Image.fromarray(img.astype(np.uint8), mode='L')
    # img.save(r'/home/ds/DataScience/Datasets/Passport/MRZ/passport_ocr/train10k/1111.png')

    return img


def gen_mask_for_back(h, w):
    n_holes = random.randint(4, 50)
    mask = np.ones((h, w), np.float32) * random.randint(30, 120)

    for n in range(n_holes):
        y = np.random.randint(h)
        x = np.random.randint(w)

        hole_size = random.randint(10, 50)
        y1 = np.clip(y - hole_size // 2, 0, h)
        y2 = np.clip(y + hole_size // 2, 0, h)
        x1 = np.clip(x - hole_size // 2, 0, w)
        x2 = np.clip(x + hole_size // 2, 0, w)

        mask[y1: y2, x1: x2] = random.randint(15, 160)

    # img = Blur(blur_limit=10, p=1.0)(image=mask)['image']
    # img = ndimage.gaussian_filter(mask, 5)
    img = ndimage.uniform_filter(mask, size=15)

    img = Image.fromarray(img.astype(np.uint8), mode='L')
    # img.save(r'/home/ds/DataScience/Datasets/Passport/MRZ/passport_ocr/train10k/1111.png')

    return img


def rotate_image(img, angle, p=0.5):
    if random.random() < p:
        newX, newY = img.shape[1], img.shape[0]
        newX, newY = int(abs(np.sin(radians(angle)) * newY) + abs(np.cos(radians(angle)) * newX)), int(
            abs(np.sin(radians(angle)) * newX) + abs(np.cos(radians(angle)) * newY))

        cx, cy = newX // 2, newY // 2
        rot_mat = cv2.getRotationMatrix2D((cx, cy), -angle, 1.0)
        rot_mat[0, 2] += (newX - img.shape[1]) / 2
        rot_mat[1, 2] += (newY - img.shape[0]) / 2

        img = cv2.warpAffine(img, rot_mat, dsize=(newX, newY))

    return img

def rotate_image_with_warp(img, angle, warpCoeff = 1.0):
    newX, newY = int(img.shape[1] * warpCoeff), img.shape[0]
    newX, newY = int(abs(np.sin(radians(angle)) * newY) + abs(np.cos(radians(angle)) * newX)), int(
        abs(np.sin(radians(angle)) * newX) + abs(np.cos(radians(angle)) * newY))

    cx, cy = newX // 2, newY // 2
    rot_mat = cv2.getRotationMatrix2D((cx, cy), -angle, 1.0)
    rot_mat[0, 2] += (newX - img.shape[1]) / 2
    rot_mat[1, 2] += (newY - img.shape[0]) / 2

    pts1 = np.float32([[0, 0], [100, 0],           [0, 100]])
    pts2 = np.float32([[0, 0], [100*warpCoeff, 0], [0, 100]])
    matrix = cv2.getAffineTransform(pts1, pts2)

    rot_mat = matrix * rot_mat

    img_rot = cv2.warpAffine(img, rot_mat, dsize=(newX, newY))

    return img_rot

def paste_image(img, img_overlay, x, y):
    # Image ranges
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])
    # Overlay ranges
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    img[y1:y2, x1:x2] = img_overlay[y1o:y2o, x1o:x2o]



def pepper(img,scale=1, p=0.5):
    if random.random() < p:

        n_holes = random.randint(1, 10)

        h = img.shape[0]
        w = img.shape[1]

        for n in range(n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            hole_w = int(np.random.triangular(1, 5, 6)*scale)  # random.randint(1, 5)
            hole_h = int(np.random.triangular(1, 5, 6)*scale)  # random.randint(1, 5)
            fill_value = random.randint(0, 160)

            y1 = y - hole_h // 2
            y2 = y1 + hole_h
            x1 = x - hole_w // 2
            x2 = x1 + hole_w

            img[y1: y2, x1: x2, :] = fill_value

    return img


def text_cutout(img, scale=1, n_holes_max = 50, p=0.5):
    if random.random() < p:

        # n_holes = random.randint(5, 50)

        n_holes = int(np.random.triangular(1, n_holes_max, n_holes_max+1))

        h = img.shape[0]
        w = img.shape[1]

        for n in range(n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            hole_w = int(np.random.triangular(1, 5, 6)*scale)
            hole_h = int(np.random.triangular(1, 5, 6)*scale)
            fill_value = random.randint(0, 50)

            y1 = y - hole_h // 2
            y2 = y1 + hole_h
            x1 = x - hole_w // 2
            x2 = x1 + hole_w

            # img[y1: y2, x1: x2, 0:3] = 255
            img[y1: y2, x1: x2, 3] = fill_value

    return img


def text_mask(img, p=0.5):
    if random.random() < p:
        loc = max(np.random.normal(loc=0.9, scale=0.1), 0.7)
        scale = max(np.random.normal(loc=0.3, scale=0.15), 0.05)
        img_scale = np.random.randint(2, 5)
        a = np.random.normal(loc=loc, scale=scale, size=(img.shape[0] // img_scale, img.shape[1] // img_scale))
        # a = ndimage.gaussian_filter(a, 1)

        a[a > 1] = 1
        a[a <= 0] = 0
        a = cv2.resize(a, (img.shape[1], img.shape[0]))
        # a = a.astype(np.uint8)

        a = a * img[:, :, 3]
        img[:, :, 3] = a.astype(np.uint8)

    return img


def downscale(img, scale, interpolation=cv2.INTER_NEAREST, p = 0.5):
    if random.random() < p:
        # h, w = img.shape[:2]

        # need_cast = interpolation != cv2.INTER_NEAREST and img.dtype == np.uint8
        # if need_cast:
        #     img = to_float(img)
        img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=interpolation)
        # upscaled = cv2.resize(downscaled, (w, h), interpolation=interpolation)
        # if need_cast:
        #     upscaled = from_float(np.clip(upscaled, 0, 1), dtype=np.dtype("uint8"))


    return  img






# Check Augmentation

In [0]:
from augs_for_ctc_mobile import *
import numpy as np
import cv2



test_image = "/home/pashkin/emeters/energy_meters_converted/img/meter0001.jpg"
save_dir = 'home/pashkin/emeters/energy_meters_converted/meter0001/'

img = test_image


img = cv2.imread(test_image,1)


#-------------------------------------------
#working well take one
img=pepper(img)

name = pepper.__name__

name_im = "test_" + f"{name}" + ".png"

img = cv2.imwrite(name_im,img)
img
print(name)
img.save('home/pashkin/emeters/energy_meters_converted/meter0001/1.png')
#----------------------------------------------


In [0]:
# aug_list = ["pepper", "blur", "wrap" ]
# treshold = 0.8

# for augs in aug_list:

#   model.fit()
  
#   if epochs == 10 and val_loss >= treshold:
#     pass



pepper
blur
wrap
